## Experiment 1

In [1]:
# CUDA check
import torch
print(torch.cuda.is_available())

True


### Custom Tokenizer creation

In [2]:
# from transformers import AutoTokenizer
# import pandas as pd
# import os

# # Load Base Tokenizer
# base_model_path = "pythainlp/wangchanglm-7.5B-sft-enth" 
# print("Loading base tokenizer...")
# tokenizer = AutoTokenizer.from_pretrained(base_model_path)

#### custum words

In [3]:
# # read the old file
# with open("custom_vocab_5w1h.txt", "r", encoding="utf-8") as file:
#     lines = file.readlines()

# # delete duplicated words and rearrange
# unique_lines = sorted(set(line.strip() for line in lines if line.strip()))

# print("\n----- words after preprocessing -----")
# for line in unique_lines:
#     print(line)

In [4]:
# # save file
# with open("custom_vocab_5w1h.txt", "w", encoding="utf-8") as file:
#     for line in unique_lines:
#         file.write(line + "\n")

# print("\nNew words saved")

In [5]:
# # load new words
# with open("custom_vocab_5w1h.txt", "r", encoding="utf-8") as f:
#     custom_tokens = [line.strip() for line in f.readlines() if line.strip()]

# # add new words to tokenizer
# num_added = tokenizer.add_tokens(custom_tokens)
# print(f"✅ Added {num_added} new tokens.")

# # save new tokenizer 
# tokenizer.save_pretrained("custom_tokenizer_5w1h")

### Step0: Load dataset 

In [6]:
import pandas as pd

df = pd.read_csv("Updated_Datasets.csv")
df.head(5)

,FilleName,Type,Input_Sec1,Input_Sec2,Input_Sec3,Full-Input,who,what,when,where,why,how,Title,Output_Sec1
0,5,กระดาษเขียนข่าวร่วม (ทท.),สศท.สปท. กําหนดจัดงานสัปดาห์วิชาการ สปท. ประจํ...,เพื่อให้การดําเนินงานเป็นไปด้วยความเรียบร้อย ส...,NaN,สศท.สปท. กําหนดจัดงานสัปดาห์วิชาการ สปท. ประจํ...,สศท.สปท.,ขอเชิญ คณะทํางานฯ เข้าร่วมประชุมเพื่อเตรียมการ...,ในวันที่ 7 พ.ค. 67 เวลา 0900,ณ ห้อง 432 ชั้น 3 อาคารอเนกประสงค์ สปท.,สศท.สปท. กําหนดจัดงานสัปดาห์วิชาการ สปท. ประจํ...,NaN,ขอเชิญเข้าร่วมประชุมเพื่อเตรียมการจัดงานสัปดาห...,สศท.สปท. ขอเชิญผู้แทนหน่วยเข้าร่วมประชุมเพื่อเ...
1,10,กระดาษเขียนข่าวร่วม (ทท.),ผบ.ทสส. กรุณาประกาศเกียรติคุณชมเชยให้กับผู้บัง...,เพื่อให้การดําเนินการเป็นไปด้วยความเรียบร้อย ก...,ขอให้หน่วยกรุณาแจ้ง ยศ-ชื่อ-สกุล ตําแหน่ง และห...,ผบ.ทสส. กรุณาประกาศเกียรติคุณชมเชยให้กับผู้บัง...,กพ.ทหาร,ขอเรียนเชิญ หน.ส่วนราชการ หรือผู้แทน เข้ารับมอ...,20 พ.ค. 67 เวลา 1000,ณ ห้องประชุม บก.ทท. ชั้น 4 อาคาร บก.ทท.,ผบ.ทสส. กรุณาประกาศเกียรติคุณชมเชยให้กับผู้บัง...,ขอให้หน่วยกรุณาแจ้ง ยศ-ชื่อ-สกุล ตําแหน่ง และห...,ขอเรียนเชิญเข้ารับมอบใบประกาศนียบัตรเกียรติคุณ...,เนื่องด้วย กพ.ทหาร ขอเรียนเชิญ ผบ.รร.ชท. เข้าร...
2,12,กระดาษเขียนข่าวร่วม (ทท.),สศท.สปท. กําหนดจัดงานสัปดาห์วิชาการ สปท. ประจํ...,เพื่อให้การดําเนินการเป็นไปด้วยความเรียบร้อย ส...,รายละเอียดเพิ่มเติมประสาน ร.ต. ภูริณัฐ เบญจขัน...,สศท.สปท. กําหนดจัดงานสัปดาห์วิชาการ สปท. ประจํ...,สศท.สปท.,"ขอเรียนเชิญ หน.นขต.บก.ทท., รอง ผบ.สปท., เสธ.สป...",ในวันที่ 4-5 มิ.ย. 67,ณ อาคาร อเนกประสงค์ สปท.,สศท.สปท. กําหนดจัดงานสัปดาห์วิชาการ สปท. ประจํ...,ขอให้ตอบแบบตอบรับการเข้าร่วมงาน ตาม QR Code 2 ...,ขอเรียนเชิญร่วมพิธีเปิด-ปิดฯ งานสัปดาห์วิชาการ...,สศท.สปท. ขอเรียนเชิญร่วมพิธีเปิด-ปิดฯ งานสัปดา...
3,18,กระดาษเขียนข่าวร่วม (ทท.),บก.ทท. (สส.ทหาร) ได้ทําสัญญาซื้อขายสิ่งอุปกรณ์...,เพื่อให้การดําเนินการเป็นไปด้วยความเรียบร้อย ส...,ขอมอบหมายให้ พ.ต. ชัชวัชร น้อยนะวะกุล ตําแหน่ง...,บก.ทท. (สส.ทหาร) ได้ทําสัญญาซื้อขายสิ่งอุปกรณ์...,สส.ทหาร,ขออนุญาตให้เจ้าหน้าที่ สส.ทหาร และเจ้าหน้าที่บ...,ตั้งแต่ 18-26 เม.ย. 67 รวม 9 วัน,พื้นที่บางซ่อน และ รร.ชท.สปท.,บก.ทท. (สส.ทหาร) ได้ทําสัญญาซื้อขายสิ่งอุปกรณ์...,NaN,ขออนุญาตเข้าพื้นที่ รร.ชท.,สส.ทหาร ขออนุญาตให้เจ้าหน้าที่ของ สส.ทหาร และเ...
4,21,กระดาษเขียนข่าวร่วม (ทท.),สปท. โดย สภท.บก.สปท. ดําเนินการจัดการเรียนการส...,เพื่อให้การดําเนินการเป็นไปด้วยความเรียบร้อย ส...,NaN,สปท. โดย สภท.บก.สปท. ดําเนินการจัดการเรียนการส...,สภท.บก.สปท.,ได้จัดทําแบบสอบถาม เรื่องการทดสอบภาษาอังกฤษ แล...,ภายในวันที่ 15 พ.ค. 67,NaN,สปท. โดย สภท.บก.สปท. ดําเนินการจัดการเรียนการส...,ขอให้กําลังพลทุกนาย ในชั้นยศ ส.ต.-พ.อ.(พ) ดําเ...,จัดการเรียนการสอนภาษาอังกฤษให้แก่กําลังพล บก.ทท.,สภท.บก.สปท. ดําเนินการจัดการเรียนการสอนภาษาอัง...


### Step1: Load Custom Tokenizer

In [7]:
from transformers import AutoTokenizer

# ✅ Load Custom Tokenizer
tokenizer_path = "./custom_tokenizer_5w1h" 
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

print("Custom Tokenizer loaded successfully.")
print("Vocab size:", len(tokenizer))


d:\NLP projectV2\.venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Custom Tokenizer loaded successfully.
Vocab size: 355761


#### Tokenization Test

In [8]:
def tokenize_and_check(text):
    if not isinstance(text, str):
        return None  # หรือ จะ return {"original_text": None, "tokens": [], "num_tokens": 0} ก็ได้
    encoding = tokenizer(text, add_special_tokens=False, return_tensors=None)
    token_ids = encoding['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    return {
        "original_text": text,
        "tokens": tokens,
        "num_tokens": len(tokens)
    }


In [9]:
# Tokenize using Custom Tokenizer
def tokenize_and_check(text):
    if not isinstance(text, str):
        return None  # หรือ จะ return {"original_text": None, "tokens": [], "num_tokens": 0} ก็ได้
    encoding = tokenizer(text, add_special_tokens=False, return_tensors=None)
    token_ids = encoding['input_ids']
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    return {
        "original_text": text,
        "tokens": tokens,
        "num_tokens": len(tokens)
    }

tokenized_results_input1 = df['Input_Sec1'].apply(tokenize_and_check)

for idx, result in enumerate(tokenized_results_input1):
    print(f"\n==== Record {idx+1} ====")
    print(f"Original Text: {result['original_text'][:]}")  
    print(f"Number of Tokens: {result['num_tokens']}")
    print(f"Tokens: {result['tokens']}")
    if idx >= 2:
        break


==== Record 1 ====
Original Text: สศท.สปท. กําหนดจัดงานสัปดาห์วิชาการ สปท. ประจําปีงบประมาณ พ.ศ. 2567 ในหัวข้อ “นวัตกรรมเพื่อความมั่นคง และความปลอดภัยสาธารณะ” ระหว่างวันที่ 4-5 มิ.ย. 67 ณ สปท. โดยมีวัตถุประสงค์ เพื่อเผยแพร่นวัตกรรมเพื่อความมั่นคง และเสริมสร้างภาพลักษณ์ที่ดีของกองทัพและหน่วยงานที่เกี่ยวข้อง รวมทั้งส่งเสริมความร่วมมือและความสัมพันธ์อันดีระหว่างหน่วยงานทั้งภายใน กห. และภายนอก กห. ซึ่งมีผู้เข้าชมงาน ประกอบด้วย กําลังพล สปท. ผู้เข้ารับการอบรมหลักสูตรในความรับผิดชอบของ สปท. นักเรียนทหาร คณาจารย์ นิสิตนักศึกษาของสถาบันการศึกษาที่มีความร่วมมือทางวิชาการระหว่างกันและบุคคลทั่วไป 

Number of Tokens: 105
Tokens: ['สศท.สปท.', '▁', 'กําหนด', 'จัดงาน', 'สัปดาห์', 'วิชาการ', '▁', 'สปท.', '▁', 'ประจำปีงบประมาณ', '▁', 'พ.ศ.', '▁', '2567', '▁', 'ในหัวข้อ', '▁“', 'นวัตกรรม', 'เพื่อ', 'ความมั่นคง', '▁', 'และ', 'ความปลอดภัย', 'สาธารณะ', '▁”', '▁', 'ระหว่างวันที่', '▁', '4-5', '▁', 'มิ.ย.', '▁', '67', '▁', 'ณ', '▁', 'สปท.', '▁', 'โดยมีวัตถุประสงค์', '▁', 'เพื่อ', 'เผยแพร่', 'นวัตกรรม', 'เพื

In [10]:
tokenized_results_input2 = df['Input_Sec2'].apply(tokenize_and_check)

for idx, result in enumerate(tokenized_results_input2):
    print(f"\n==== Record {idx+1} ====")
    print(f"Original Text: {result['original_text'][:]}")  
    print(f"Number of Tokens: {result['num_tokens']}")
    print(f"Tokens: {result['tokens']}")
    if idx >= 2:
        break


==== Record 1 ====
Original Text: เพื่อให้การดําเนินงานเป็นไปด้วยความเรียบร้อย สศท.สปท. จึงขอเชิญ คณะทํางานฯ เข้าร่วมประชุมเพื่อเตรียมการจัดงานฯ ในวันที่ 7 พ.ค. 67 เวลา 0900 ณ ห้อง 432 ชั้น 3 อาคารอเนกประสงค์ สปท. โดยมี พล.ท. สมยศ ฉันทวรลักษณ์ เสธ.สปท. เป็นประธานการประชุมฯ รายละเอียดตาม QR Code ด้านล่าง ทั้งนี้ ได้มอบหมายให้ ร.ท. ชัชรินทร์ เลิศยศบดินทร์ ประจําแผนกแผน และโครงการ กศษ.สศท.สปท. โทรศัพท์เคลื่อนที่ 08 2342 4267 เป็นผู้ประสานรายละเอียดการปฏิบัติ
Number of Tokens: 98
Tokens: ['เพื่อให้', 'การดําเนินงาน', 'เป็นไป', 'ด้วย', 'ความเรียบร้อย', '▁', 'สศท.สปท.', '▁', 'จึงขอเชิญ', '▁', 'คณะทํางาน', 'ฯ', '▁', 'เข้าร่วมประชุม', 'เพื่อ', 'เตรียมการ', 'จัดงาน', 'ฯ', '▁', 'ในวันที่', '▁', '7', '▁', 'พ.ค.', '▁', '67', '▁', 'เวลา', '▁', '09', '00', '▁', 'ณ', '▁', 'ห้อง', '▁', '432', '▁', 'ชั้น', '▁', '3', '▁', 'อาคารอเนกประสงค์', '▁', 'สปท.', '▁', 'โดยมี', '▁', 'พล.ท.', '▁', 'สมยศ', '▁', 'ฉันทวรลักษณ์', '▁', 'เสธ.สปท.', '▁', 'เป็น', 'ประธานการประชุม', 'ฯ', '▁', 'รายละเอียด', 'ตาม', '▁', 'QR

### Step 2: Preprocessing

In [11]:
import re

def preprocess_text(text):
    # ตัดแบ่งประโยคเบื้องต้น (จุด, เว้นวรรค)
    sentences = re.split(r'(?<=[.])\s+', text)
    sentences = [s.strip() for s in sentences if s.strip()]
    
    # Boost Formality เล็กน้อย: แก้คำไม่เป็นทางการ (ตัวอย่าง)
    replacements = {
        "ช่วย": "กรุณา",
        "ดูแล": "ดำเนินการตรวจสอบ",
        "ดู": "ตรวจสอบ",
        "บอก": "แจ้ง",
        "ให้": "อนุเคราะห์",
    }
    boosted_sentences = []
    for sentence in sentences:
        for informal, formal in replacements.items():
            sentence = re.sub(rf'\b{informal}\b', formal, sentence)
        boosted_sentences.append(sentence)
    
    return " ".join(boosted_sentences)

df['Input_Sec1'] = df['Input_Sec1'].apply(preprocess_text)

for idx, row in df.iterrows():
    print(f"\n🧹 ตัวอย่าง Record {idx+1}")
    # print(f"Original Text:\n{row['combined_input'][:300]}...\n")
    print(f"Preprocessed Text:\n{row['Input_Sec1'][:300]}...")
    if idx >= 2:
        break


🧹 ตัวอย่าง Record 1
Preprocessed Text:
สศท.สปท. กําหนดจัดงานสัปดาห์วิชาการ สปท. ประจําปีงบประมาณ พ.ศ. 2567 ในหัวข้อ “นวัตกรรมเพื่อความมั่นคง และความปลอดภัยสาธารณะ” ระหว่างวันที่ 4-5 มิ.ย. 67 ณ สปท. โดยมีวัตถุประสงค์ เพื่อเผยแพร่นวัตกรรมเพื่อความมั่นคง และเสริมสร้างภาพลักษณ์ที่ดีของกองทัพและหน่วยงานที่เกี่ยวข้อง รวมทั้งส่งเสริมความร่วมมือ...

🧹 ตัวอย่าง Record 2
Preprocessed Text:
ผบ.ทสส. กรุณาประกาศเกียรติคุณชมเชยให้กับผู้บังคับบัญชา จํานวน 4 นาย จากการดูแลใส่ใจผู้ใต้บังคับบัญชา โดยให้ความช่วยเหลือและสนับสนุน เพื่อให้บุตรของผู้ใต้บังคับบัญชา ได้เข้ารับการศึกษาในสถาบันที่ต้องการ และให้ กพ.ทหาร และ สลก.บก.ทท. จัดพิธีมอบใบประกาศเกียรติคุณ ชมเชย ก่อนการประชุม หน.ส่วนราชการ บก.ทท...

🧹 ตัวอย่าง Record 3
Preprocessed Text:
สศท.สปท. กําหนดจัดงานสัปดาห์วิชาการ สปท. ประจําปีงบประมาณ พ.ศ. 2567 หัวข้อ “นวัตกรรมเพื่อความมั่นคงและความปลอดภัยสาธารณะ” ในวันที่ 4-5 มิ.ย. 67 ณ อาคาร อเนกประสงค์ สปท. โดยมีวัตถุประสงค์เพื่อเผยแพร่ความรู้ด้านนวัตกรรมความมั่นคง และเสริมสร้าง ภาพลักษณ์ที่ดีของกองทัพและหน่ว

In [12]:
df['Input_Sec2'] = df['Input_Sec2'].apply(preprocess_text)

for idx, row in df.iterrows():
    print(f"\n🧹 ตัวอย่าง Record {idx+1}")
    # print(f"Original Text:\n{row['combined_input'][:300]}...\n")
    print(f"Preprocessed Text:\n{row['Input_Sec2'][:300]}...")
    if idx >= 2:
        break


🧹 ตัวอย่าง Record 1
Preprocessed Text:
เพื่อให้การดําเนินงานเป็นไปด้วยความเรียบร้อย สศท.สปท. จึงขอเชิญ คณะทํางานฯ เข้าร่วมประชุมเพื่อเตรียมการจัดงานฯ ในวันที่ 7 พ.ค. 67 เวลา 0900 ณ ห้อง 432 ชั้น 3 อาคารอเนกประสงค์ สปท. โดยมี พล.ท. สมยศ ฉันทวรลักษณ์ เสธ.สปท. เป็นประธานการประชุมฯ รายละเอียดตาม QR Code ด้านล่าง ทั้งนี้ ได้มอบหมายให้ ร.ท. ชั...

🧹 ตัวอย่าง Record 2
Preprocessed Text:
เพื่อให้การดําเนินการเป็นไปด้วยความเรียบร้อย กพ.ทหาร ขอเรียนเชิญ หน.ส่วนราชการ หรือผู้แทน เข้ารับมอบใบประกาศฯ ตามวัน เวลา และสถานที่และเชิญผู้แทนหน่วยร่วมซักซ้อมพิธีมอบใบประกาศฯ ใน 20 พ.ค. 67 เวลา 1000 ณ ห้องประชุม บก.ทท. ชั้น 4 อาคาร บก.ทท....

🧹 ตัวอย่าง Record 3
Preprocessed Text:
เพื่อให้การดําเนินการเป็นไปด้วยความเรียบร้อย สศท.สปท. ขอเรียนเชิญ หน.นขต.บก.ทท., รอง ผบ.สปท., เสธ.สปท., รอง เสธ.สปท. (1), รอง เสธ.สปท. (2), หน.นขต.สปท., หน.นขต.บก.สปท., หน.นขต.สศท.สปท. และกําลังพล บก.ทท. เข้าร่วมในพิธี เปิด-ปิด และร่วมชมงานฯ โดยขอให้ตอบแบบตอบรับการเข้าร่วมงาน ตาม QR Code 2 ภายใน 28 ...


#### Filtering

In [13]:
import re

# Expand abbreviation function
def expand_abbreviations(text, abbreviation_dict):
    for abbr, full_name in abbreviation_dict.items():
        pattern = r'\b' + re.escape(abbr) + r'\b'
        text = re.sub(pattern, f"{full_name} ({abbr})", text)
    return text

# Example abbreviation dictionary
abbreviation_dict = {
    "ก.ค.": "กรกฎาคม",
    "ก.พ.": "กุมภาพันธ์",
    "ก.ย.": "กันยายน",
    "กกท.ศทท.สส.ทหาร": "กองการโทรคมนาคม ศูนย์การโทรคมนาคมทหาร กรมการสื่อสารทหาร",
    "กกล.บก.สปท.": "กองกลาง กองบัญชาการ สถาบันวิชาการป้องกันประเทศ",
    "กขส.ยบ.ทหาร": "กองขนส่ง ยุทธบริการทหาร",
    "กตป.สปช.ทหาร": "กองติดตามและประเมินผล สำนักงานปลัดบัญชีทหาร",
    "กตส.สตป.": "กองตรวจสอบ สำนักงานตรวจสอบภายใน",
    "กทด.บก.สปท.": "กองทดสอบ กองบัญชาการ สถาบันวิชาการป้องกันประเทศ",
    "กทพ.กพ.ทหาร": "กองทัพพิเศษ กองกำลังพลทหาร",
    "กนผ.กร.ทหาร": "กองนโยบายและแผน กรมกิจการพลเรือนหาร",
    "กนผ.สนพ.กพ.ทหาร": "กองนโยบายและแผน สำนักงานนโยบายและแผน กองกำลังพลทหาร",
    "กพ.ทหาร": "กองกำลังพลทหาร",
    "กมศ.บก.สปท.": "กองมาตรฐานการศึกษา กองบัญชาการ สถาบันวิชาการป้องกันประเทศ",
    "กร.ทหาร": "กรมกิจการพลเรือนทหาร",
    "สบ.ทหาร": "กรมสารบรรณทหาร",
}


In [14]:
# ขยายตัวย่อ
df['Input_Sec2'] = df['Input_Sec2'].apply(lambda x: expand_abbreviations(x, abbreviation_dict))
df['Input_Sec1'] = df['Input_Sec1'].apply(lambda x: expand_abbreviations(x, abbreviation_dict))

In [15]:
def create_prompt_fewshot(input_sec2, input_sec1):
    prompt = f"""
    คุณคือผู้ช่วยสรุปหนังสือราชการ
    ให้อ่านข้อมูลที่กำหนด และสรุปเนื้อหาอย่างกระชับ ชัดเจน และเป็นภาษาทางการ
    ห้ามแต่งข้อมูลเพิ่มจากที่กำหนด
    ให้นำเสนอเป็นข้อความย่อความเท่านั้น ไม่ต้องจัดหมวดหมู่ ไม่ต้องตอบเป็น JSON

    ข้อความใหม่สำหรับสกัด:
    \"\"\"{input_sec2}\"\"\"

    เหตุผลเบื้องหลัง:
    \"\"\"{input_sec1}\"\"\"

    โปรดตอบเฉพาะสรุปข้อความย่อเป็นภาษาไทยเท่านั้น
    """
    return prompt


### Step 3: Experiment

#### Abstractive Mode: Ollama

In [6]:
# Config สำหรับ Ollama
Ollama_API_URL = "http://localhost:11434/api/chat"
Ollama_Model_Name = "wangchanglm"

In [7]:
import requests

def query_ollama_chat(prompt):
    payload = {
        "model": Ollama_Model_Name,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        response = requests.post(Ollama_API_URL, json=payload, timeout=120)
        response.raise_for_status()
        result = response.json()
        return result['message']['content']
    except Exception as e:
        print(f"⚠️ Error calling Ollama: {e}")
        return None


#### Extractive Mode

In [19]:
from transformers import AutoModel

# Config WangchanBERTa (สำหรับ extractive)
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [20]:
def get_sentence_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().numpy()

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

def extract_sentences(title, input_sec1, content, top_k=3):
    if pd.isnull(title) or pd.isnull(content):
        return ""
    anchor_text = title + " " + input_sec1
    sentences = re.split(r'(?<=[.!?])\s+', content)
    sentences = [s.strip() for s in sentences if s.strip()]
    if not sentences:
        return ""
    anchor_emb = get_sentence_embedding(anchor_text)
    sentence_embs = [get_sentence_embedding(sent) for sent in sentences]
    sims = [cosine_similarity(anchor_emb, sent_emb)[0][0] for sent_emb in sentence_embs]
    top_indices = sorted(range(len(sims)), key=lambda i: sims[i], reverse=True)[:top_k]
    selected_sentences = [sentences[i] for i in top_indices]
    return ' '.join(selected_sentences)

In [22]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

def rouge_l_score(ref, pred):
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    return scorer.score(ref, pred)['rougeL'].fmeasure

def bleu_score(ref, pred):
    smoothie = SmoothingFunction().method4
    return sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothie)

#### Process

In [23]:
# EXPERIMENT_MODE = ["extractive", "abstractive"]

In [24]:
from tqdm import tqdm

extractive_outputs = []
abstractive_outputs = []
human_outputs = df['Output_Sec1'].tolist()

for idx, row in tqdm(df.iterrows(), total=len(df)):
    title = row['Title']
    input_sec1 = row['Input_Sec1']
    input_sec2 = row['Input_Sec2']

    # Extractive V2
    extracted = extract_sentences(title, input_sec1, input_sec2)
    extractive_outputs.append(extracted)

    # Abstractive V2
    input_sec2 = row['Input_Sec2']
    prompt = create_prompt_fewshot(input_sec2, input_sec1)
    # prompt = create_prompt_abstractive(input_sec2)
    summary = query_ollama_chat(prompt)
    if summary:
        abstractive_outputs.append(summary)
    else:
        abstractive_outputs.append("")

    # # Abstractive V2.1
    # summary = generate_summary_openthaigpt(input_sec2)
    # abstractive_outputs.append(summary)


100%|██████████| 31/31 [05:57<00:00, 11.53s/it]


### Step5: Evaluation

In [12]:
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

similarity_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2') 

extractive_rouge = []
abstractive_rouge = []
extractive_bleu = []
abstractive_bleu = []

extractive_cosine = []
abstractive_cosine = []

for human, extr, abstr in tqdm(zip(human_outputs, extractive_outputs, abstractive_outputs), total=len(human_outputs)):
    # --- ROUGE-L ---
    extractive_rouge.append(rouge_l_score(human, extr))
    abstractive_rouge.append(rouge_l_score(human, abstr))

    # --- BLEU ---
    extractive_bleu.append(bleu_score(human, extr))
    abstractive_bleu.append(bleu_score(human, abstr))

    # --- Cosine Similarity ---
    emb_human = similarity_model.encode(human)
    emb_extr = similarity_model.encode(extr)
    emb_abstr = similarity_model.encode(abstr)

    cosine_extr = cosine_similarity([emb_human], [emb_extr])[0][0]
    cosine_abstr = cosine_similarity([emb_human], [emb_abstr])[0][0]

    extractive_cosine.append(cosine_extr)
    abstractive_cosine.append(cosine_abstr)


d:\NLP projectV2\.venv310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'human_outputs' is not defined

In [26]:
df_result = pd.DataFrame({
    "Human_Reference": human_outputs,
    "Extractive_Output": extractive_outputs,
    "Abstractive_Output": abstractive_outputs,
    "Extractive_ROUGE_L": extractive_rouge,
    "Abstractive_ROUGE_L": abstractive_rouge,
    "Extractive_BLEU": extractive_bleu,
    "Abstractive_BLEU": abstractive_bleu,
    "Extractive_Cosine": extractive_cosine,
    "Abstractive_Cosine": abstractive_cosine
})


df_result.head()

,Human_Reference,Extractive_Output,Abstractive_Output,Extractive_ROUGE_L,Abstractive_ROUGE_L,Extractive_BLEU,Abstractive_BLEU,Extractive_Cosine,Abstractive_Cosine
0,สศท.สปท. ขอเชิญผู้แทนหน่วยเข้าร่วมประชุมเพื่อเ...,จึงขอเชิญ คณะทํางานฯ เข้าร่วมประชุมเพื่อเตรียม...,สศท.สปท. กำหนดจัดงานสัปดาห์วิชาการสปท. ในหัวข้...,0.250000,0.571429,0.059419,0.377006,0.828602,0.831551
1,เนื่องด้วย กพ.ทหาร ขอเรียนเชิญ ผบ.รร.ชท. เข้าร...,เพื่อให้การดําเนินการเป็นไปด้วยความเรียบร้อย ก...,กองกำลังพลทหาร (กพ.ทหาร) เชิญ หน.ส่วนราชการ หร...,0.200000,0.923077,0.149420,0.299269,0.827930,0.824840
2,สศท.สปท. ขอเรียนเชิญร่วมพิธีเปิด-ปิดฯ งานสัปดา...,เข้าร่วมในพิธี เปิด-ปิด และร่วมชมงานฯ โดยขอให้...,สศท.สปท. จัดงานสัปดาห์วิชาการสปท. ประจําปีงบปร...,0.000000,0.615385,0.000000,0.111026,0.581678,0.827434
3,สส.ทหาร ขออนุญาตให้เจ้าหน้าที่ของ สส.ทหาร และเ...,เพื่อให้การดําเนินการเป็นไปด้วยความเรียบร้อย ส...,สถานะการดำเนินการซื้อ-ขายอุปกรณ์และการปําหมีน ...,0.857143,0.555556,0.298567,0.013362,0.935565,0.648653
4,สภท.บก.สปท. ดําเนินการจัดการเรียนการสอนภาษาอัง...,ขอให้กําลังพลทุกนาย ในชั้นยศ ส.ต.-พ.อ.(พ) ดําเ...,สำหรับการดำเนินการเรียนการสอนภาษาอังกฤษ และการ...,0.000000,0.000000,0.014936,0.008812,0.603229,0.786960


In [28]:
output_save_path = "Experiment1_Result.xlsx"
df_result.to_excel(output_save_path, index=False)

print(f"\n✅ All processing done. Result saved to {output_save_path}")


✅ All processing done. Result saved to Experiment1_Result.xlsx


In [29]:
# df.to_excel('output.xlsx', index=False, engine='openpyxl')

### Auto Select Best Summary

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

def select_best(input_sec1, input_sec2, human_reference):
    # --- 1. Extractive Summarization ---
    extractive_summary = extract_sentences(None, input_sec1, input_sec2)

    # --- 2. Abstractive Summarization ---
    prompt = create_prompt_fewshot(input_sec2, input_sec1)
    abstractive_summary = query_ollama_chat(prompt)
    if abstractive_summary is None:
        abstractive_summary = ""

    # --- 3. Calculate Cosine Similarity ---
    emb_human = similarity_model.encode(human_reference)
    emb_extr = similarity_model.encode(extractive_summary)
    emb_abstr = similarity_model.encode(abstractive_summary)

    cosine_extr = cosine_similarity([emb_human], [emb_extr])[0][0]
    cosine_abstr = cosine_similarity([emb_human], [emb_abstr])[0][0]

    # --- 4. Select Best ---
    if cosine_abstr >= cosine_extr:
        best_summary = abstractive_summary
        best_method = "Abstractive"
        best_cosine = cosine_abstr
    else:
        best_summary = extractive_summary
        best_method = "Extractive"
        best_cosine = cosine_extr

    return best_summary, best_method, best_cosine


In [34]:
# ✅ Example Usage

# สมมติ input
input_sec1 = "ผบ.ทสส. (จก.ยบ.ทหาร รับคําสั่ง) กรุณาอนุมัติให้ ยบ.ทหาร (สนพ.ยบ.ทหาร) ในฐานะหัวหน้าสายวิทยาการแพทย์ บก.ทท. ดําเนินการเปิดการฝึกอบรมหลักสูตรนายทหารประทวน สายวิทยาการแพทย์ รุ่นที่ 3 ประจําปีงบประมาณ พ.ศ. 2568 ระหว่าง 7 พ.ย. 20 ธ.ค. 27 โดยมีผู้เข้ารับ การฝึกอบรมฯ จํานวน 12 นาย ณ สนพ.ยบ.ทหาร (บางซ่อน) นั้น"  # Section 1 (preprocessed)
input_sec2 = "ยบ.ทหาร (สนพ.) ในฐานะหัวหน้าสายวิทยาการแพทย์ บก.ทท. จึงขอเรียนเชิญ หน.หน่วย หรือผู้แทน และผู้เข้ารับการฝึกอบรมฯ เข้าร่วมพิธีเปิดการฝึกอบรมหลักสูตรนายทหารประทวนสายวิทยาการแพทย์ รุ่นที่ 3 ประจําปีงบประมาณ พ.ศ. 2568 โดยมี จก.ยบ.ทหาร เป็นประธานฯ ในวันพุธที่ 5 พฤศจิกายน พ.ศ. 2567 เวลา 1300 ณ ห้องประชุมหทัยนเรศ ชั้น 2 สนพ.ยบ.ทหาร (บางซ่อน) การแต่งกาย เครื่องแบบปกติคอพับแขนยาว (ทอ. อินทรธนูแข็ง) งดหมวก และสําหรับผู้เข้ารับการฝึกอบรมฯ หลังพิธีเปิด เข้ารับฟังการชี้แจง เตรียมความพร้อม ก่อนการฝึกอบรมฯ "  # Section 2 (prepared text)
human_reference = "ยบ.ทหาร (สนพ.ยบ.ทหาร) กําหนดพิธีเปิดการฝึกอบรมหลักสูตรนายทหารประทวน สายวิทยาการแพทย์ รุ่นที่ 3 ประจําปีงบประมาณ พ.ศ. 2568 ในวันพุธที่ 5 พ.ย. 67 เวลา 1300 ณ ห้องประชุม หทัยนเรศ ชั้น 2 สนพ.ยบ.ทหาร (บางซ่อน) โดยมี จก.ยบ.ทหาร เป็นประธานฯ การแต่งกาย เครื่องแบบปกติ คอพับแขนยาว (ทอ. อินทรธนูแข็ง) งดหมวก รายละเอียดตามสิ่งที่ส่งมาด้วย "  
# Human-written reference summary

final_summary, selected_method, final_cosine = select_best(input_sec1, input_sec2, human_reference)


print("✅ Final Summary (After Postprocess):", final_summary)
print("Selected Method:", selected_method)
print("Semantic Similarity (Cosine):", final_cosine)

✅ Final Summary (After Postprocess): พ.ศ. 2568 ในฐานะหัวหน้าสายวิทยากรแพทย์ บก.ทส. จึงขอเรียนเชิญผู้เข้ารับการฝึกอบรม 12 นาย ในหลักสูตรนายทหารประทวนสายวิทยากรแพทย์ 3 ต่อรุ่น ผู้ช่วยศึกษาจะได้เครื่องแบบปกติคอฟับแขนยาว (ทอ.อินทรธนูแข็ง) ในวันพุธที่ 5 พฤศจิกายน พ.ศ. 2567 ณ สนพ.ยบ.ทหาร (บางซ่อน)
    "ผบ.ทสส. อนุมัติให้"
Selected Method: Abstractive
Semantic Similarity (Cosine): 0.8220937


## Experiment 2

In [16]:
# สร้างข้อความข้อ 2 (Fact) จากข้อ 1 แบบ Fixed Template

def create_fixed_fact(input_sec1):
    fixed_fact = ""
    return fixed_fact


In [9]:
def create_prompt_for_section3(input_sec1, fixed_fact, with_feedback=False):
    base_prompt = f"""
<fact>
ข้อ ๑: {input_sec1}

ข้อ ๒: {fixed_fact}
</fact>

<instruction>
โปรดเขียนข้อเสนอหรือสั่งการ (ข้อ ๓) โดยปฏิบัติตามแนวทางต่อไปนี้:
- ลำดับเป็นข้อ ๆ: 2.1, 2.2, 2.3
- ห้ามเกิน 3 ข้อ
- แต่ละข้อเป็นประโยคคำสั่งสั้น ๆ (1–2 บรรทัด)
- ใช้ภาษาราชการที่สุภาพ และสอดคล้องกับข้อ ๑ และข้อ ๒
- ห้ามมีข้อความสรุปท้าย
</instruction>
"""

    if with_feedback:
        feedback_example = """
<example>
2.1 เรียนเชิญ ผบ.รร.ชท. หรือผู้แทนเข้าร่วมพิธีเปิดฯ ตามข้อ 1
2.2 กสน.ฯ จัดรถรับ-ส่ง เข้าร่วมพิธีเปิดฯ
2.3 ผธก.ฯ บันทึกลงระบบสารบรรณอิเล็กทรอนิกส์ (ECM) ให้หน่วยที่เกี่ยวข้องดำเนินการต่อไป
</example>
"""
        base_prompt = feedback_example.strip() + "\n" + base_prompt.strip()

    return base_prompt.strip()


In [10]:
import numpy as np

def generate_best_section3(input_sec1, human_reference_section3, with_feedback=False):
    """
    Generate ข้อ 3 และคืนค่า: (Best Summary, Best Score)
    """
    # ✅ สร้างข้อ 2 จากข้อ 1
    fixed_fact = create_fixed_fact(input_sec1)

    # ✅ เตรียม Prompt
    prompt = create_prompt_for_section3(input_sec1, fixed_fact, with_feedback=with_feedback)

    generated_texts = []
    scores = []

    for _ in range(3):  # ยิง 3 ครั้ง
        gen_text = query_ollama_chat(prompt)
        if gen_text:
            generated_texts.append(gen_text)
            emb_human = similarity_model.encode(human_reference_section3)
            emb_generated = similarity_model.encode(gen_text)
            score = cosine_similarity([emb_human], [emb_generated])[0][0]
            scores.append(score)

    if not generated_texts:
        return "", 0.0

    best_idx = np.argmax(scores)
    best_text = generated_texts[best_idx]
    best_score = scores[best_idx]

    # (Optional) Postprocess
    # polished_text = rewrite_summary_wangchan(best_text)

    return best_text, best_score


In [15]:
# ตัวอย่างสมมติ
input_sec1 = "ยบ.ทหาร (สนพ.ยบ.ทหาร) กําหนดพิธีเปิดการฝึกอบรมหลักสูตรนายทหารประทวน สายวิทยาการแพทย์ รุ่นที่ 3 ประจําปีงบประมาณ พ.ศ. 2568 ในวันพุธที่ 5 พ.ย. 67 เวลา 1300 ณ ห้องประชุม หทัยนเรศ ชั้น 2 สนพ.ยบ.ทหาร (บางซ่อน) โดยมี จก.ยบ.ทหาร เป็นประธานฯ การแต่งกาย เครื่องแบบปกติ คอพับแขนยาว (ทอ. อินทรธนูแข็ง) งดหมวก"

human_reference_section3 = "" \
"2.1 เรียนเชิญ ผบ.รร.ชท. หรือผู้แทนเข้าร่วมพิธีเปิดฯ ตามข้อ 1"
"2.2 กสน.ฯ จัดรถรับ-ส่ง เข้าร่วมพิธีเปิดฯ"
"2.3 ผธก.ฯ บันทึกลงระบบสารบรรณอิเล็กทรอนิกส์ (ECM) ให้หน่วยที่เกี่ยวข้องดําเนินการต่อไป" \
""

# ✅ Generate แบบมี Feedback Correction
section3_with_feedback, score_with_feedback = generate_best_section3(
    input_sec1, human_reference_section3, with_feedback=True
)

# ✅ Generate แบบไม่มี Feedback Correction
section3_without_feedback, score_without_feedback = generate_best_section3(
    input_sec1, human_reference_section3, with_feedback=False
)


# ✅ ดูผล
print("\n✅ Section 3 (With Feedback Correction):")
print(section3_with_feedback)
print("Cosine Similarity:", score_with_feedback)

print("\n✅ Section 3 (Without Feedback Correction):")
print(section3_without_feedback)
print("Cosine Similarity:", score_without_feedback)



✅ Section 3 (With Feedback Correction):
2.1 ผู้ผนึกเชิญ ผบ.รร.ชท. หรือผู้แทนจัดตั้งพิธีเปิดในวันพุธที่ 5 พ.ย. 67, 1300 ณ ห้องประชุม หทัยนเรศ ชั้น 2
2.2 กสน.ฯ จัดรถรับ-ส่ง ผู้ผนึกในพิธีเปิด
2.3 ผธก.ฯ บันทึกลงระบบ SAP ให้หน่วยที่เกี่ยวข้องดำเนินการต่อไป
Cosine Similarity: 0.73151875

✅ Section 3 (Without Feedback Correction):
2.1 จัดทำในช่วงวันพุธที่ 5 พ.ย. 67 เรียกว่าสัญญาณให้เคยอบรมหลักสูตรนายทหารประทวน สายวิทยาการแพทย์ รุ่นที่ 3 แผนกรบกฎบ้าน
2.2 ค้นหาเอกสารและสื่อทั้งหมดที่จำเป็นในการจัดตัวฝึกอบรม
2.3 แผนการจัดข่าวสารและสื่อไปที่สถานีและสื่อสารทั้งหมด
Cosine Similarity: 0.5051496
